In [1]:
import tensorflow as tf
import numpy as np

각 파라미터와 API 정리 필요

In [8]:
class Seq2Seq(object):
    
    def __init__(self,
               source_vocab_size,
               target_vocab_size,
               buckets,
               size,
               num_layers,
               max_gradient_norm,
               batch_size,
               learning_rate,
               learning_rate_decay_factor,
               use_lstm=False,
               num_samples=512,
               forward_only=False,
               dtype=tf.float32):
        # buckets = [(5,10), (10,15), (20,25), (40,50)]
        
        self.source_vocab_size = source_vocab_size
        self.target_vocab_size = target_vocab_size
        self.buckets = buckets
        self.batch_size = batch_size
        self.learning_rate = tf.Variable(
            float(learning_rate), trainable=False, dtype=dtype)
        self.learning_rate_decay_op = self.learning_rate.assign(
            self.learning_rate * learning_rate_decay_factor)
        self.global_step = tf.Variable(0, trainable=False)
        
        # sampled softmax를 사용하려면 output_projection이 필요하다
        output_projection = None
        softmax_loss_function = None
        # Sampled softmax only makes sense if we sample less than vocabulary size.
        # 샘플의 갯수가 target_vocab_size보다 작은 경우에만 사용한다.
        # 더 큰 경우에는 표준 소프트맥스 손실함수가 낫다.
        
        if num_samples > 0 and num_samples < self.target_vocab_size:
            w_t = tf.get_variable("proj_w", [self.target_vocab_size, size], dtype=dtype)
            w = tf.transpose(w_t)
            b = tf.get_variable("proj_b", [self.target_vocab_size], dtype=dtype)
            output_projection = (w, b)
            
            def sampled_loss(inputs, labels):
                labels = tf.reshape(labels, [-1, 1])
                # We need to compute the sampled_softmax_loss using 32bit floats to
                # avoid numerical instabilities.
                local_w_t = tf.cast(w_t, tf.float32)
                local_b = tf.cast(b, tf.float32)
                local_inputs = tf.cast(inputs, tf.float32)
                
                return tf.cast(
                        tf.nn.sampled_softmax_loss(local_w_t, local_b, local_inputs, labels,
                                                   num_samples, self.target_vocab_size),dtype)
            
            softmax_loss_function = sampled_loss

            """
            weight matrix와 bias vector의 쌍으로 이루어진 출력 투영을 구성한다.
            이것이 사용되는 경우, rnn cell은 (batch size x target_vocab_size)
            대신 (batch size x size) 형태의 벡터를 반환한다.
            로짓을 복원하려면 가중치 행렬로 곱한 뒤 편향 벡터를 더해야 한다. 
            """
            

            # Create the internal multi-layer cell for our RNN.
        single_cell = tf.nn.rnn_cell.GRUCell(size)
        if use_lstm:
            single_cell = tf.nn.rnn_cell.BasicLSTMCell(size)
        cell = single_cell
        if num_layers > 1:
            cell = tf.nn.rnn_cell.MultiRNNCell([single_cell] * num_layers)

        # The seq2seq function: we use embedding for the input and attention.
        def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
              return tf.nn.seq2seq.embedding_attention_seq2seq(
                  encoder_inputs,
                  decoder_inputs,
                  cell,
                  num_encoder_symbols=source_vocab_size,
                  num_decoder_symbols=target_vocab_size,
                  embedding_size=size,
                  output_projection=output_projection,
                  feed_previous=do_decode,
                  dtype=dtype)
        
        """
        encoder_inputs, decoder_inputs은 이산값을 나타내는 정수 벡터
        이 입력은 밀도 높은 표현으로 임베드되는데 이 임베딩을 구성하기 위해
        심볼들의 숫자들(num_encoder_symbols, num_decoder_symbols)을 지정할
        필요가 있다.

        * feed_previous : False이면 decoder_inputs 텐서를 그대로 사용,
        True면 decoder_inputs의 첫 번째 원소만 사용한다? 리스트의 다른 텐서는
        모두 무시되고, 인코더의 이전 결과를 대신 사용

        * output_projection : 이 인자가 지정되지 않으면, 임베딩 모델의 출력은
        생성된 각 심볼의 로짓을 나타내는 (batch size x num_decoder_symbols)
        형태의 텐서로 나오게 되는데, 이렇게 되면 (num_decoder_symbols)가 클 때
        실용적이지 않게 된다. 

        먼저 작은 출력 텐서를 반환하고 이후에 output_projection을 이용해서 큰 출력
        텐서로 projection하는 편이 낫다. 이렇게 해서 sampled softmax loss와 함께
        사용할 수 있다.

        """

        # bucket과 padding 사용

        # Feeds for inputs.
        self.encoder_inputs = []
        self.decoder_inputs = []
        self.target_weights = []
        for i in range(buckets[-1][0]):  # Last bucket is the biggest one.
            self.encoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                    name="encoder{0}".format(i)))
        for i in range(buckets[-1][1] + 1):
            self.decoder_inputs.append(tf.placeholder(tf.int32, shape=[None],
                                                    name="decoder{0}".format(i)))
            self.target_weights.append(tf.placeholder(dtype, shape=[None],
                                                    name="weight{0}".format(i)))

        # Our targets are decoder inputs shifted by one.
        targets = [self.decoder_inputs[i + 1]
                   for i in range(len(self.decoder_inputs) - 1)]
        
             # Training outputs and losses.
        if forward_only:
            self.outputs, self.losses = tf.nn.seq2seq.model_with_buckets(
              self.encoder_inputs, self.decoder_inputs, targets,
              self.target_weights, buckets, lambda x, y: seq2seq_f(x, y, True),
              softmax_loss_function=softmax_loss_function)
            # If we use output projection, we need to project outputs for decoding.
            if output_projection is not None: # 여기서 로짓을 복원?
                for b in range(len(buckets)):
                    self.outputs[b] = [
                    tf.matmul(output, output_projection[0]) + output_projection[1]
                    for output in self.outputs[b]
                    ]
        else:
            self.outputs, self.losses = tf.nn.seq2seq.model_with_buckets(
              self.encoder_inputs, self.decoder_inputs, targets,
              self.target_weights, buckets,
              lambda x, y: seq2seq_f(x, y, False),
              softmax_loss_function=softmax_loss_function)

        # Gradients and SGD update operation for training the model.
        params = tf.trainable_variables()
        if not forward_only:
            self.gradient_norms = []
            self.updates = []
            opt = tf.train.GradientDescentOptimizer(self.learning_rate)
            for b in range(len(buckets)):
                gradients = tf.gradients(self.losses[b], params)
                clipped_gradients, norm = tf.clip_by_global_norm(gradients,
                                                             max_gradient_norm)
                self.gradient_norms.append(norm)
                self.updates.append(opt.apply_gradients(
                zip(clipped_gradients, params), global_step=self.global_step))

        self.saver = tf.train.Saver(tf.all_variables())

In [9]:
model = Seq2Seq()

TypeError: __init__() missing 9 required positional arguments: 'source_vocab_size', 'target_vocab_size', 'buckets', 'size', 'num_layers', 'max_gradient_norm', 'batch_size', 'learning_rate', and 'learning_rate_decay_factor'